In [2]:
import papermill as pm
import sys
import os
import os.path as op
from glob import glob
import pandas as pd
from datetime import timedelta
sys.path.append('../scripts/')
from mod import list_data, hub_projects

In [3]:
repos = list_data()
repos.head()

created_at              2018   2019
org        repo                    
jupyter    nbconvert     NaN   41.0
           notebook      NaN  411.0
           repo2docker   NaN  176.0
jupyterhub binder        NaN   39.0
           binderhub     NaN   68.0

In [4]:
date = "{:%Y-%m-%d}".format(pd.datetime.today() - timedelta(days=30))
path_this_report = '../reports/{}'.format(date)


In [5]:
import shutil as sh
import os
if os.path.exists(path_this_report):
    sh.rmtree(path_this_report)

In [7]:
for org, repos in hub_projects.items():
    print(org)
    for repo in repos:
        print('   '+repo)
        this_ntbk_params = '{}-{}'.format(org, repo)
        out_folder = '../reports/{}/{}/{}/'.format(date, org, repo)
        if not op.isdir(out_folder):
            os.makedirs(out_folder)
        for template in glob('./templates/*.ipynb'):
            if 'summary.ipynb' in template:
                continue
            name = op.basename(template)
            new_name = op.join(out_folder, name)
            pm.execute_notebook(template, new_name, parameters={'org': org, 'repo': repo, 'date': date}, cwd='./templates/', report_mode=True)

jupyter
   repo2docker



jupyterhub
   zero-to-jupyterhub-k8s



   the-littlest-jupyterhub



   jupyterhub



   binderhub


...


   mybinder.org-deploy



   configurable-http-proxy



   nativeauthenticator



   traefik-proxy



   jupyter-server-proxy


In [8]:
# Collect the results of each notebook and make a summary notebook
_ = pm.execute_notebook('./templates/summary.ipynb', op.join(out_folder.rsplit(os.sep, 3)[0], 'summary.ipynb'), parameters={'date': date}, cwd='./templates/', report_mode=True)

In [9]:
!pandoc ../reports/2019-02-20/summary.ipynb -o ../test.html

# Jupyter Book

## Generate the TOC

(still a WIP)

In [89]:
out_notebooks = ['../reports/{}/summary.ipynb'.format(date)] + glob('../reports/{}/**/**/*.ipynb'.format(date))
toc = []
for ii, ifile in enumerate(out_notebooks):
    ifile = ifile.replace('.ipynb', '')
    if ii == 0:
        # Summary will be first
        toc.append({'title': date, "url": ifile.split('reports'+os.sep)[-1], 'sections': []})
        continue
     
    ######################################################
    # Update Jupyter Book to check for {{ cell.metadata['jupyter']['source_hidden'] = True }} in each cell. If so, add a "hidden" param
    ######################################################
    
    name = ' '.join(ifile.rsplit(os.sep, 3)[-3:]).replace('.ipynb', '')
    toc[-1]['sections'].append({'title': name, 'url': ifile.split('reports'+os.sep)[-1]})


In [90]:
import shutil as sh

In [92]:
import nbformat as nbf

In [116]:
ntbk = nbf.read('../reports/2019-01-18/jupyter/repo2docker/comments_commits.ipynb', nbf.NO_CONVERT)

In [120]:
ntbk['cells'][1].metadata['jupyter']['source_hidden'] = True

KeyError: 'jupyter'

In [102]:
for ii in ntbk['cells']:
    print(ii['metadata'])

{'papermill': {'duration': 0.021794, 'end_time': '2019-02-18T03:18:50.725319', 'exception': False, 'start_time': '2019-02-18T03:18:50.703525', 'status': 'completed'}, 'tags': []}
{'papermill': {'duration': 0.542844, 'end_time': '2019-02-18T03:18:51.291706', 'exception': False, 'start_time': '2019-02-18T03:18:50.748862', 'status': 'completed'}, 'tags': []}
{'papermill': {'duration': 0.033622, 'end_time': '2019-02-18T03:18:51.357736', 'exception': False, 'start_time': '2019-02-18T03:18:51.324114', 'status': 'completed'}, 'tags': ['parameters']}
{'papermill': {'duration': 0.028936, 'end_time': '2019-02-18T03:18:51.414614', 'exception': False, 'start_time': '2019-02-18T03:18:51.385678', 'status': 'completed'}, 'tags': ['injected-parameters']}
{'papermill': {'duration': 0.036194, 'end_time': '2019-02-18T03:18:51.473552', 'exception': False, 'start_time': '2019-02-18T03:18:51.437358', 'status': 'completed'}, 'tags': []}
{'papermill': {'duration': 0.021864, 'end_time': '2019-02-18T03:18:51.52

In [91]:
from ruamel.yaml import YAML
yaml = YAML()
with open('../reports/book/_data/toc.yml', 'w') as ff:
    yaml.dump(toc, ff)

path_content = '../reports/book/content/{}'.format(date)
if op.isdir(path_content):
    sh.rmtree(path_content)
sh.copytree('../reports/{}'.format(date), path_content)

'../reports/book/content/2019-01-18'